<a href="https://colab.research.google.com/github/w-lly/Agentic-AI-for-Insights/blob/Task-1-pdf-scanning/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [15]:
%pip install pymupdf
%pip install pandas
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 999 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 286 kB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 19.6 MB/s eta 0:00:011
     |████████████████████████████████| 161 kB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 40.5 MB/s eta 0:00:011
     |███████

# Step 1: Extract text from the PDF using PyMuPDF

In [21]:
import fitz
import pymupdf
import pandas as pd

doc = pymupdf.open("data/input/20169320_TUM_SB1200_en.pdf")
print(doc.name)
print(doc.page_count)
extracted_text = ""
for page in doc:
    extracted_text += page.get_text()
# print(extracted_text)
# Save to CSV
df = pd.DataFrame({"text": [extracted_text]})
df.to_csv("data/output/extracted_text_from_pdf.csv", index=False)

data/input/20169320_TUM_SB1200_en.pdf
88


# Step 2: Chunk text with metadata

In [23]:
import tiktoken
import pandas as pd

# Load tokenizer 
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
print(encoding.name)

# Chunking function
def chunk_text_with_metadata(text, chunk_size=500, overlap=50):
    tokens = encoding.encode(text)
    chunks = []
    start = 0
    chunk_id = 0

    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = encoding.decode(chunk_tokens)
        
        chunks.append({
            "chunk_id": chunk_id,
            "start_token": start,
            "end_token": end,
            "text": chunk_text
        })

        chunk_id += 1
        start += chunk_size - overlap  # sliding window
    return pd.DataFrame(chunks)

chunk_df = chunk_text_with_metadata(extracted_text)

chunk_df.to_csv("data/output/chunked_pdf_text.csv", index=False).head(10)

cl100k_base


AttributeError: 'NoneType' object has no attribute 'head'

In [18]:
df= pd.read_csv("data/output/chunked_pdf_text.csv")
df

,chunk_id,start_token,end_token,text
0,0,0,500,USER MANUAL\nSB1200\nFCS4074\nen\nRead the use...
1,1,450,950,44\n7.11\nEmptying the cooling unit\n 45\n8\nP...
2,2,900,1400,.4\nLong-term decommissioning\n 82\n13.5\nTran...
3,3,1350,1850,intended exclusively for operation indoors.\n...
4,4,1800,2300,or add-on units if you are not familiar with ...
5,5,2250,2750,"is permanently assembled, contact the service..."
6,6,2700,3200,leading to injury.\na) Never insert any objec...
7,7,3150,3650,\nDescription\nClean components using the 5-st...
8,8,3600,4100,.3\nDepiction of error messages\nEspresso\nCap...
9,9,4050,4550,for fixing are included)\n560.0623.576\nFrank...
